# Setup

In [1]:
import pandas as pd
import dataclasses
import nba_api

## nba_api packages

In [22]:
from nba_api.stats.static.teams import get_teams, find_teams_by_nickname
#from nba_api.stats.static.players import 

from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import BoxScoreTraditionalV2
from nba_api.stats.endpoints import BoxScorePlayerTrackV2

In [10]:
# Get Spurs' team info
all_teams = get_teams()
spurs_info = find_teams_by_nickname('spurs') # find spurs nickname
spurs_id = spurs_info[0]['id'] # get spurs team ID

# Define functions

In [86]:
def combine_shooting_fields(df, makes_col, att_col, field_name):
    df[field_name] = df[makes_col].astype(int).astype(str) + '-' + df[att_col].astype(int).astype(str)

In [61]:
def format_names(name):
    split_names = name.split(' ')
    first_name = split_names[0][0] + '.'
    
    split_names[0] = first_name
    
    return ' '.join(split_names)

# Get Spurs' box scores

In [16]:
# find all Spurs games
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=spurs_id,
                                              season_nullable='2019-20')
team_box_scores_df = gamefinder.get_data_frames()[0]
team_box_scores_df.head(n=10)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22019,1610612759,SAS,San Antonio Spurs,0021901314,2020-08-13,SAS @ UTA,L,239,112,...,0.667,9,35,44,24,4,8,15,20,-6.0
1,22019,1610612759,SAS,San Antonio Spurs,0021901298,2020-08-11,SAS vs. HOU,W,240,123,...,0.900,9,50,59,28,10,5,22,20,18.0
2,22019,1610612759,SAS,San Antonio Spurs,0021901287,2020-08-09,SAS @ NOP,W,241,122,...,0.969,15,36,51,22,12,6,17,27,9.0
3,22019,1610612759,SAS,San Antonio Spurs,0021901274,2020-08-07,SAS vs. UTA,W,240,119,...,0.769,8,39,47,27,10,5,13,18,8.0
4,22019,1610612759,SAS,San Antonio Spurs,0021901264,2020-08-05,SAS vs. DEN,L,239,126,...,0.852,10,25,35,29,10,5,7,20,-6.0
5,22019,1610612759,SAS,San Antonio Spurs,0021901254,2020-08-03,SAS @ PHI,L,240,130,...,0.897,13,31,44,21,7,6,14,23,-2.0
6,22019,1610612759,SAS,San Antonio Spurs,0021901246,2020-08-02,SAS @ MEM,W,241,108,...,0.727,12,37,49,22,5,6,16,20,2.0
7,22019,1610612759,SAS,San Antonio Spurs,0021901237,2020-07-31,SAS vs. SAC,W,240,129,...,0.815,9,33,42,32,6,4,15,17,9.0
8,12019,1610612759,SAS,San Antonio Spurs,0011900130,2020-07-28,SAS @ IND,W,241,118,...,0.905,7,41,48,29,8,8,12,18,7.0
9,12019,1610612759,SAS,San Antonio Spurs,0011900115,2020-07-25,SAS vs. BKN,L,240,119,...,0.857,12,45,57,21,4,11,24,32,-5.0


In [21]:
spurs_games = list(team_box_scores_df.GAME_ID.unique()) # grab unique list of game IDs

In [24]:
player_box_scores = BoxScorePlayerTrackV2(game_id=spurs_games[0])
player_box_scores_df = player_box_scores.get_data_frames()[0]

In [26]:
player_box_scores_df.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
       'PLAYER_NAME', 'START_POSITION', 'COMMENT', 'MIN', 'SPD', 'DIST',
       'ORBC', 'DRBC', 'RBC', 'TCHS', 'SAST', 'FTAST', 'PASS', 'AST', 'CFGM',
       'CFGA', 'CFG_PCT', 'UFGM', 'UFGA', 'UFG_PCT', 'FG_PCT', 'DFGM', 'DFGA',
       'DFG_PCT'],
      dtype='object')

In [36]:
bench_stats = BoxScoreTraditionalV2(game_id=spurs_games[0]).data_sets

In [46]:
bench_stats[2].get_data_frame()

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,STARTERS_BENCH,MIN,FGM,FGA,FG_PCT,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS
0,0021901314,1610612762,Jazz,UTA,Utah,Starters,70:43,12,32,0.375,...,1.000,3,15,18,8,1,3,3,4,33
1,0021901314,1610612762,Jazz,UTA,Utah,Bench,169:17,35,69,0.507,...,0.875,9,16,25,17,6,2,5,19,85
2,0021901314,1610612759,Spurs,SAS,San Antonio,Starters,153:06,29,56,0.518,...,0.632,5,25,30,17,3,3,12,14,77
3,0021901314,1610612759,Spurs,SAS,San Antonio,Bench,83:37,10,23,0.435,...,0.733,4,9,13,7,1,5,3,5,34


# Get Player Box Scores

In [101]:
player_box_scores_df = BoxScoreTraditionalV2(game_id=spurs_games[0]).data_sets[0].get_data_frame()
player_box_scores_df.fillna(0, inplace=True)
player_box_scores_df.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0021901314,1610612759,SAS,San Antonio,1629640,Keldon Johnson,F,,30:11,8.0,...,0.0,2.0,2.0,0.0,1.0,0.0,3.0,4.0,24.0,4.0
1,0021901314,1610612759,SAS,San Antonio,1629677,Luka Samanic,F,,31:27,5.0,...,1.0,5.0,6.0,3.0,0.0,1.0,1.0,4.0,16.0,7.0
2,0021901314,1610612759,SAS,San Antonio,1627751,Jakob Poeltl,C,,21:08,5.0,...,2.0,3.0,5.0,3.0,1.0,2.0,1.0,3.0,10.0,-1.0
3,0021901314,1610612759,SAS,San Antonio,1629022,Lonnie Walker IV,G,,34:04,6.0,...,1.0,2.0,3.0,4.0,1.0,0.0,1.0,0.0,15.0,-12.0
4,0021901314,1610612759,SAS,San Antonio,1627749,Dejounte Murray,G,,36:16,5.0,...,1.0,13.0,14.0,7.0,0.0,0.0,6.0,3.0,12.0,4.0


In [96]:
player_box_scores_df.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
       'PLAYER_NAME', 'START_POSITION', 'COMMENT', 'MIN', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS'],
      dtype='object')

In [102]:
player_box_scores_df.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
       'PLAYER_NAME', 'START_POSITION', 'COMMENT', 'MIN', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS'],
      dtype='object')

In [108]:
combine_shooting_fields(player_box_scores_df, 'FGM', 'FGA', 'FG')
combine_shooting_fields(player_box_scores_df, 'FG3M', 'FG3A', '3P')
combine_shooting_fields(player_box_scores_df, 'FTM', 'FTA', 'FT')

In [111]:
player_box_scores_df.to_json('player_box_score_prototype.json', orient='records')